In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/housing/housing.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.ocean_proximity.value_counts()

In [ ]:
df2 = pd.get_dummies(df, columns= ['ocean_proximity'])
df2.head()

In [ ]:
df2.info()

In [ ]:
sns.heatmap(df.corr(), annot= True)

In [ ]:
df.hist(figsize=(15,12))

In [ ]:
df2.isnull().sum()

In [ ]:
df2['total_bedrooms'] = df2['total_bedrooms'].fillna(df2['total_bedrooms'].mean())
df2.isnull().sum()

In [ ]:
df2 = df2.drop(['longitude', 'latitude'], axis = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
x = df2.drop('median_house_value', axis = 1)
y = df2['median_house_value']

In [ ]:
x_scaled = sc.fit_transform(x.values)
y_scaled = sc.fit_transform(y.values.reshape(-1, 1)).flatten()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size = 0.33, random_state = 1)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_test)
y_pred

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
## Test Score
r2_score(y_test, y_pred)

In [ ]:
## Train Score
r2_score(y_train, lr.predict(x_train))

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mae

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr, x_train, y_train, cv = 10)

In [ ]:
## Test Score
cv_score = r2_score(y_test, lr.predict(x_test))
cv_score

In [ ]:
lr.coef_

In [ ]:
pd.DataFrame(lr.coef_, index= x.columns, columns= ['Coefficients']).sort_values(ascending = False, by = 'Coefficients')

In [ ]:
from sklearn.linear_model import Ridge
rr = Ridge()

In [ ]:
rr.fit(x_train, y_train)

In [ ]:
## Train Score
r2_score(y_train, rr.predict(x_train))

In [ ]:
## Test Score
ridge_score = r2_score(y_test, rr.predict(x_test))
ridge_score

In [ ]:
rr.coef_

In [ ]:
pd.DataFrame(rr.coef_, index= x.columns, columns= ['Coefficients']).sort_values(ascending = False, by = 'Coefficients')

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtree = DecisionTreeRegressor()
dtree.fit(x_train, y_train)

In [ ]:
## Train Score
r2_score(y_train, dtree.predict(x_train))

In [ ]:
## since we got 99%. It seems to be overfit hence we will do cross validation
cross_val_score(dtree, x_train, y_train, cv=10)

In [ ]:
## Test Score
dtree_score = r2_score(y_test, dtree.predict(x_test))
dtree_score

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [ ]:
rf.fit(x_train, y_train)

In [ ]:
## Train Score
r2_score(y_train, rf.predict(x_train))

In [ ]:
## since we got 94%. It seems to be overfit hence we will do cross validation
cross_val_score(rf, x_train, y_train, cv=10)

In [ ]:
## Test Score
rf_score = r2_score(y_test, rf.predict(x_test))
rf_score

In [ ]:
algorithm = [cv_score, ridge_score, dtree_score, rf_score]
index = ['Cross Validation','Ridge Regression', 'Decision Tree', 'Random Forest']
pd.DataFrame(algorithm, index=index, columns=['Scores']).sort_values(ascending = False, by=['Scores'])